## 연습문제

In [176]:
import numpy as np 
import pandas as pd
import seaborn as sns

### 1. Iris - sns.dataset('iris')

In [177]:
iris = sns.load_dataset("iris")

In [178]:
iris.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [179]:
# 원본 데이터
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


#### a. 붓꽃 종 별로 꽃잎길이, 꽃잎 폭, 꽃받침길이, 꽃받침폭의 평균, 표준편차 등 기초통계량을 구하시오.

In [180]:
df1 = iris.groupby(iris.species).agg(['mean', 'std'])
df1

sepal_length           sepal_width           petal_length  \
                   mean       std        mean       std         mean   
species                                                                
setosa            5.006  0.352490       3.428  0.379064        1.462   
versicolor        5.936  0.516171       2.770  0.313798        4.260   
virginica         6.588  0.635880       2.974  0.322497        5.552   

                     petal_width            
                 std        mean       std  
species                                     
setosa      0.173664       0.246  0.105386  
versicolor  0.469911       1.326  0.197753  
virginica   0.551895       2.026  0.274650

#### b. 3분위수(Q3)와 1분위수(Q1)의 차이보다 1.5배가 크거나 작은 데이터는 이상치이다. 
- Q1 - 1.5 * (Q3 - Q1) 보다 작은 데이터
- Q3 + 1.5 * (Q3 - Q1) 보다 큰 데이터
#### 이상치를 제거하고 위의 4가지 항목에 대해서 평균, 표준편차를 구하시오.

In [181]:
df2 = iris.copy()
df2.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [182]:
def get_new_data(x):
    q1 = x.quantile(.25)
    q3 = x.quantile(.75)
    iqr = q3 - q1
    low_boundary = q1 - 1.5 * iqr
    high_boundary = q3 + 1.5 * iqr
    x[(x < low_boundary) | (x > high_boundary)] = np.NaN
    outlier = x.value_counts().sum() != 50
    return outlier, x.mean(), x.std()

In [183]:
iris_spec = ['setosa', 'versicolor', 'virginica']
iris_stat = ['sepal_length','sepal_width','petal_length','petal_width']

In [184]:
species = []; items = []; old_mean_list = []; old_std_list = []
for i in iris_spec:
    for k in iris_stat:
        s = df2[df2.species == i][k]
        species.append(i); items.append(k)
        old_mean_list.append(s.mean()); old_std_list.append(s.std())

In [185]:
data = {
    'old_mean': old_mean_list,
    'old_std': old_std_list
}
index = [species, items]
df3 = pd.DataFrame(data, index=index)
df3

old_mean   old_std
setosa     sepal_length     5.006  0.352490
           sepal_width      3.428  0.379064
           petal_length     1.462  0.173664
           petal_width      0.246  0.105386
versicolor sepal_length     5.936  0.516171
           sepal_width      2.770  0.313798
           petal_length     4.260  0.469911
           petal_width      1.326  0.197753
virginica  sepal_length     6.588  0.635880
           sepal_width      2.974  0.322497
           petal_length     5.552  0.551895
           petal_width      2.026  0.274650

In [186]:
outlier_list = []; new_mean_list = []; new_std_list = []
for i in iris_spec:
    for k in iris_stat:
        s = df2[df2.species == i][k]
        outlier, new_mean, new_std = get_new_data(s)
        outlier_list.append(outlier); new_mean_list.append(new_mean); new_std_list.append(new_std)

In [187]:
df3['is_outlier'] = outlier_list
df3['new_mean'] = mean_list
df3['new_std'] = std_list
df3

old_mean   old_std  is_outlier  new_mean   new_std
setosa     sepal_length     5.006  0.352490       False  5.006000  0.352490
           sepal_width      3.428  0.379064        True  3.431250  0.320343
           petal_length     1.462  0.173664        True  1.460870  0.129062
           petal_width      0.246  0.105386        True  0.233333  0.085883
versicolor sepal_length     5.936  0.516171       False  5.936000  0.516171
           sepal_width      2.770  0.313798       False  2.770000  0.313798
           petal_length     4.260  0.469911        True  4.285714  0.437798
           petal_width      1.326  0.197753       False  1.326000  0.197753
virginica  sepal_length     6.588  0.635880        True  6.622449  0.593459
           sepal_width      2.974  0.322497        True  2.955319  0.260275
           petal_length     5.552  0.551895       False  5.552000  0.551895
           petal_width      2.026  0.274650       False  2.026000  0.274650

### 2. Titanic - sns.load_dataset('titanic')

In [188]:
# 원본 데이터
titanic = sns.load_dataset("titanic")
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


#### b. 타이타닉호의 승객 중 나이를 명시하지 않은 고객은 나이를 명시한 고객의 평균 나이 값이 되도록 고치시오.

In [189]:
titanic.age = titanic.age.fillna(round(titanic.age.mean(), 2))
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,29.7,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True


#### a. 타이타닉호의 승객에 대해 나이와 성별에 의한 카테고리 열인 category1 열을 만드시오.
- 20살이 넘으면 성별을 그대로 사용한다.
- 20살 미만이면 성별에 관계없이 "child" 라고 한다.

In [190]:
titanic['category1'] = titanic.apply(lambda x: x.sex if x.age >= 20 else 'child', axis=1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,29.7,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,female
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


#### c. 성별, 선실(class)별, 출발지(embark_town)별 생존율을 구하시오.

In [191]:
# 성별 생존율
df_sex = titanic.groupby(['sex', 'alive'])[['alive']].count()
df_sex['alive_ratio'] = df_sex['alive'] / df_sex.groupby('sex')['alive'].sum()
df_sex

alive  alive_ratio
sex    alive                    
female no        81     0.257962
       yes      233     0.742038
male   no       468     0.811092
       yes      109     0.188908

In [192]:
titanic[['sex', 'survived']].groupby(['sex']).mean()

,survived
sex,
female,0.742038
male,0.188908


In [193]:
# 선실별 생존율
df_class = titanic.groupby(['class', 'alive'])[['alive']].count()
df_class['alive_ratio'] = df_class['alive'] / df_class.groupby('class')['alive'].sum()
df_class

alive  alive_ratio
class  alive                    
First  no        80     0.370370
       yes      136     0.629630
Second no        97     0.527174
       yes       87     0.472826
Third  no       372     0.757637
       yes      119     0.242363

In [194]:
titanic[['class', 'survived']].groupby(['class']).mean()

,survived
class,
First,0.629630
Second,0.472826
Third,0.242363


In [195]:
# 출발지별 생존율
df_embark = titanic.groupby(['embark_town', 'alive'])[['alive']].count()
df_embark['alive_ratio'] = df_embark['alive'] / df_embark.groupby('embark_town')['alive'].sum()
df_embark

alive  alive_ratio
embark_town alive                    
Cherbourg   no        75     0.446429
            yes       93     0.553571
Queenstown  no        47     0.610390
            yes       30     0.389610
Southampton no       427     0.663043
            yes      217     0.336957

In [196]:
titanic[['embark_town', 'survived']].groupby(['embark_town']).mean()

,survived
embark_town,
Cherbourg,0.553571
Queenstown,0.389610
Southampton,0.336957


#### d. 타이타닉호 승객을 나이 그룹으로 나누고, 각 그룹별 생존율을 구하시오.

In [197]:
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "중년", "장년", "노년"]

In [198]:
titanic['age_cat'] = pd.cut(titanic.age, bins, labels=labels)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,age_cat
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male,청년
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child,미성년자
888,0,3,female,29.7,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,female,청년
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male,청년
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male,중년


In [199]:
df1 = titanic.groupby(['age_cat', 'alive'])[['alive']].count()
df1['alive_ratio'] = df1['alive'] / df1.groupby('age_cat')['alive'].sum()
df1

alive  alive_ratio
age_cat alive                    
미성년자    no        95     0.575758
        yes       70     0.424242
청년      no       271     0.665848
        yes      136     0.334152
중년      no       139     0.576763
        yes      102     0.423237
장년      no        38     0.644068
        yes       21     0.355932
노년      no         4     0.800000
        yes        1     0.200000

#### e. qcut 명령으로 세 개의 나이 그룹을 만들고, 나이 그룹별 남녀 성비와 생존율을 구하시오.

In [200]:
# 3개의 나이 그룹 ( Q1, Q2, Q3 )
cats = pd.qcut(titanic.age, 3, labels=['Q1', 'Q2', 'Q3'])
cats, pd.value_counts(cats)         # 나이 그룹과 각 그룹별 count 출력

(0      Q1
 1      Q3
 2      Q2
 3      Q3
 4      Q3
        ..
 886    Q2
 887    Q1
 888    Q2
 889    Q2
 890    Q3
 Name: age, Length: 891, dtype: category
 Categories (3, object): [Q1 < Q2 < Q3],
 Q2    304
 Q1    301
 Q3    286
 Name: age, dtype: int64)

In [201]:
titanic['cats'] = pd.qcut(titanic.age, 3, labels=['Q1', 'Q2', 'Q3'])
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,age_cat,cats
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male,청년,Q2
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child,미성년자,Q1
888,0,3,female,29.7,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,female,청년,Q2
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male,청년,Q2
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male,중년,Q3


In [202]:
df_cats = titanic.groupby(['cats', 'alive'])[['alive']].count()
df_cats['alive_ratio'] = df_cats['alive'] / df_cats.groupby('cats')['alive'].sum()
df_cats

alive  alive_ratio
cats alive                    
Q1   no       177     0.588040
     yes      124     0.411960
Q2   no       202     0.664474
     yes      102     0.335526
Q3   no       170     0.594406
     yes      116     0.405594

In [203]:
# 나이 그룹별 남녀 성비
df_sex2 = titanic.groupby(['cats', 'sex'])[['sex']].count()
df_sex2['sex_ratio'] = df_sex2['sex'] / df_sex2.groupby('cats')['sex'].sum()
df_sex2

sex  sex_ratio
cats sex                   
Q1   female  122   0.405316
     male    179   0.594684
Q2   female   97   0.319079
     male    207   0.680921
Q3   female   95   0.332168
     male    191   0.667832

In [204]:
# 나이 그룹별 남녀 성비에 따른 생존율
df_cats2 = titanic.groupby(['cats', 'sex', 'alive'])[['alive']].count()
df_cats2['alive_ratio'] = df_cats2['alive'] / df_cats2.groupby(['cats', 'sex'])['alive'].sum()
df_cats2

alive  alive_ratio
cats sex    alive                    
Q1   female no        35     0.286885
            yes       87     0.713115
     male   no       142     0.793296
            yes       37     0.206704
Q2   female no        29     0.298969
            yes       68     0.701031
     male   no       173     0.835749
            yes       34     0.164251
Q3   female no        17     0.178947
            yes       78     0.821053
     male   no       153     0.801047
            yes       38     0.198953

### 3. Mile Per Gallon - sns.load_dataset('mpg')

In [205]:
# 원본 데이터
mpg = sns.load_dataset("mpg")
mpg

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger


In [206]:
mpg.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
count,398.000000,398.000000,398.000000,392.000000,398.000000,398.000000,398.000000
mean,23.514573,5.454774,193.425879,104.469388,2970.424623,15.568090,76.010050
std,7.815984,1.701004,104.269838,38.491160,846.841774,2.757689,3.697627
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000
25%,17.500000,4.000000,104.250000,75.000000,2223.750000,13.825000,73.000000
50%,23.000000,4.000000,148.500000,93.500000,2803.500000,15.500000,76.000000
75%,29.000000,8.000000,262.000000,126.000000,3608.000000,17.175000,79.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000


#### a. 배기량(displacement) 대비 마력(horsepower) 열(hp_per_cc)을 추가하시오.

In [207]:
mpg['hp_per_cc'] = mpg['horsepower'] / mpg['displacement']
mpg.tail()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,hp_per_cc
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl,0.614286
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup,0.536082
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage,0.622222
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger,0.658333
397,31.0,4,119.0,82.0,2720,19.4,82,usa,chevy s-10,0.689076


#### b. name으로부터 manufacturer(제조사)와 모델을 추출하여 새로운 열 manufacturer와 model을 추가하고, name열은 삭제하시오.

In [208]:
list = mpg['name'].tolist()
tmp = []
# 첫 공백을 기준으로 한 번만 나눈다
for word in list:
    tmp.append(word.split(maxsplit=1))   

# 공백을 기준으로 나눈 결과를 manufacturer, model의 두 개의 열로 갖는 데이터프레임 df1 생성               
df1 = pd.DataFrame(tmp, columns=['manufacturer', 'model'])
mpg = pd.concat([mpg, df1], axis=1)                         # 옆으로 데이터 열 연결
mpg.tail(10)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,hp_per_cc,manufacturer,model
388,26.0,4,156.0,92.0,2585,14.5,82,usa,chrysler lebaron medallion,0.589744,chrysler,lebaron medallion
389,22.0,6,232.0,112.0,2835,14.7,82,usa,ford granada l,0.482759,ford,granada l
390,32.0,4,144.0,96.0,2665,13.9,82,japan,toyota celica gt,0.666667,toyota,celica gt
391,36.0,4,135.0,84.0,2370,13.0,82,usa,dodge charger 2.2,0.622222,dodge,charger 2.2
392,27.0,4,151.0,90.0,2950,17.3,82,usa,chevrolet camaro,0.596026,chevrolet,camaro
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl,0.614286,ford,mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup,0.536082,vw,pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage,0.622222,dodge,rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger,0.658333,ford,ranger
397,31.0,4,119.0,82.0,2720,19.4,82,usa,chevy s-10,0.689076,chevy,s-10


In [209]:
# 'name' 열 삭제
del mpg['name']    
mpg.tail(10)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,manufacturer,model
388,26.0,4,156.0,92.0,2585,14.5,82,usa,0.589744,chrysler,lebaron medallion
389,22.0,6,232.0,112.0,2835,14.7,82,usa,0.482759,ford,granada l
390,32.0,4,144.0,96.0,2665,13.9,82,japan,0.666667,toyota,celica gt
391,36.0,4,135.0,84.0,2370,13.0,82,usa,0.622222,dodge,charger 2.2
392,27.0,4,151.0,90.0,2950,17.3,82,usa,0.596026,chevrolet,camaro
393,27.0,4,140.0,86.0,2790,15.6,82,usa,0.614286,ford,mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,0.536082,vw,pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,0.622222,dodge,rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,0.658333,ford,ranger
397,31.0,4,119.0,82.0,2720,19.4,82,usa,0.689076,chevy,s-10


#### c. 엔진의 실린더(cylinders) 갯수별 연비(mpg)의 평균을 구하시오.

In [210]:
# 실린더 갯수별 연비의 평균을 내림차순으로 정렬
df2 = mpg.groupby('cylinders')[['mpg']].mean()
df2.sort_values(by='mpg', ascending=False)

,mpg
cylinders,
4,29.286765
5,27.366667
3,20.550000
6,19.985714
8,14.963107


#### d. 생산지(origin)별 배기량 대비 마력(hp_per_cc)의 평균을 구하시오.

In [211]:
# 생산지별 배기량 대비 마력의 평균을 내림차순으로 정렬
df3 = mpg.groupby('origin')[['hp_per_cc']].mean()
df3.sort_values(by='hp_per_cc', ascending=False)

,hp_per_cc
origin,
japan,0.789485
europe,0.740005
usa,0.512063


#### e. 모델이 5개 이상인 제조사에 대하여 연비(mpg)의 평균이 가장 좋은 제조사 Top 5를 구하시오.

In [212]:
# 모델이 5개 이상인 제조사는 True, 아니면 False
above_5 = mpg.groupby(['manufacturer'])['model'].count() >= 5
above_5

manufacturer
amc               True
audi              True
bmw              False
buick             True
cadillac         False
capri            False
chevroelt        False
chevrolet         True
chevy            False
chrysler          True
datsun            True
dodge             True
fiat              True
ford              True
hi               False
honda             True
maxda            False
mazda             True
mercedes         False
mercedes-benz    False
mercury           True
nissan           False
oldsmobile        True
opel             False
peugeot           True
plymouth          True
pontiac           True
renault           True
saab             False
subaru           False
toyota            True
toyouta          False
triumph          False
vokswagen        False
volkswagen        True
volvo             True
vw                True
Name: model, dtype: bool

In [213]:
# model이 5개 넘는 제조사들의 평균 연비
mpg_above5 = mpg.groupby('manufacturer')[['mpg']].mean()[above_5]
mpg_above5

,mpg
manufacturer,
amc,18.246429
audi,26.714286
buick,19.182353
chevrolet,20.472093
chrysler,17.266667
datsun,31.113043
dodge,22.060714
fiat,28.912500
ford,19.694118


In [214]:
# 위를 토대로 평균 연비가 높은 순서대로 Top 5 정렬
mpg_top5 = mpg_above5.sort_values(by='mpg', ascending=False)[:5]
mpg_top5

,mpg
manufacturer,
vw,39.016667
honda,33.761538
renault,32.880000
datsun,31.113043
mazda,30.860000
